In [1]:
# Import the webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
# Import the chrome client
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
# Import waiting stuff
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
# Import nba_api for shot data
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
# Import pandas for data analysis
import pandas as pd
from extras import *
pd.set_option('display.max_columns', None)

In [2]:
# Players and the tables of their individual stats

ja = Player('Ja Morant', '1629630', '1610612763', ['2019-20','2020-21','2021-22'], 'm/moranja01', ['2020', '2021', '2022'])
drose = Player('Derrick Rose', '201565', '1610612741', ['2008-09','2009-10','2010-11'], 'r/rosede01', ['2009', '2010', '2011'])
west = Player('Russell Westbrook', '201566', '1610612760', ['2008-09','2009-10','2010-11'], 'w/westbru01', ['2009', '2010', '2011'])
iver = Player('Allen Iverson', '947', '1610612755', ['1996-97','1997-98','1998-99'], 'i/iversal01', [ '1997', '1998', '1999'])
players = [ja, drose, west, iver]
players2 = [ja, drose]

# Install web-driver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Gonzalo\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


Stats of each player per 100 possessions from NBA stats and basketball reference

In [4]:
# Getting data season average per 100pos
tables_NBA_100pos = ['//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[2]/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[3]/div/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[4]/div/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[5]/div/div[3]/table', '//*[@id="__next"]/div[2]/section/div[4]/section[3]/div/div[6]/div/div[3]/table']
#driver.get('https://www.nba.com/stats/')
# Uncomment next line if first time to accept cookies and prevent errors
#WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'//*[@id="onetrust-accept-btn-handler"]'))).click()

for player in players:
    # Website
    print('Starting with', player.name())
    driver.get('https://www.nba.com/stats/player/' + player.nbaid() + '?PerMode=Per100Possessions')
    season_avg_per100 = pd.DataFrame()

    # Get all 5 tables in the webpage
    for j in range(5):
        print('Table', j+1)
        html_table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, tables_NBA_100pos[j]))).get_attribute("outerHTML") #This is the xpath (not full)
        table = pd.read_html(html_table)
        season_avg_per100 = pd.concat([season_avg_per100, table[0].tail(3).dropna(axis=1)], axis = 1)
    print('Season average per 100 possesions for', player.name(), 'completed.')
    # Format
    season_avg_per100.columns = season_avg_per100.columns.get_level_values(1)
    season_avg_per100 = season_avg_per100.loc[:,~season_avg_per100.columns.duplicated()]
    season_avg_per100.to_json('../data/'+ player.nbaid() +'/season_avg_per100.json')

    # Get data from boxscores
    for year in player.bballrefseasons():
        season_boxscore = pd.DataFrame()
        driver.get('https://www.basketball-reference.com/players/' + player.bballrefid() + '/gamelog/' + year)
        html_table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pgl_basic"]'))).get_attribute('outerHTML')
        season_boxscore = pd.read_html(html_table)[0]
        season_boxscore = season_boxscore.drop(['Rk', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7'], axis = 1).dropna()
        season_boxscore = season_boxscore.set_index('G').drop('G', axis = 0)
        season_boxscore.to_json('../data/'+ player.nbaid() +'/' + year + 'boxscores.json')
        print('Boxscore', player.name(), year , 'completed.')

Starting with Allen Iverson
Table 1
Table 2
Table 3
Table 4
Table 5
Season average per 100 possesions for Allen Iverson completed.
Boxscore Allen Iverson 1997 completed.
Boxscore Allen Iverson 1998 completed.
Boxscore Allen Iverson 1999 completed.


In [6]:
button_xpath = '//*[@id="onetrust-close-btn-container"]/button'
xpath = '//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div/div/div/div[2]/table'
urls = ['4279888/ja-morant', '3468/russell-westbrook', '3456/derrick-rose', '366/allen-iverson']
for url in urls:
    driver.get('https://www.espn.com/nba/player/advancedstats/_/id/' + url)
    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH , button_xpath))).click()
    print(pd.read_html(driver.find_element(By.XPATH, xpath).get_attribute('outerHTML'))[0].head(3)['PER'])

0    17.45
1    16.81
2    24.53
Name: PER, dtype: float64
0    15.20
1    17.90
2    23.63
Name: PER, dtype: float64
0    16.05
1    18.69
2    23.62
Name: PER, dtype: float64
0    0.0
1    0.0
2    0.0
Name: PER, dtype: float64


Drives per game data for ja

In [5]:
drives_per_game = pd.DataFrame()
# Uncomment next lines if first time to accept cookies and prevent errors
#driver.get('https://www.nba.com/stats/')
#WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'//*[@id="onetrust-accept-btn-handler"]'))).click()

for year in players[0].nbaseasons():
    # Scraping
    driver.get('https://www.nba.com/stats/players/drives/?Season='+ year + '&SeasonType=Regular%20Season&sort=DRIVES&dir=1&CF=GP*GE*30&PlayerPosition=G')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))).click()
    driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]').click()
    html_table = driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/div/table').get_attribute('outerHTML')
    # Storing
    drives_per_game = pd.read_html(html_table)[0]
    drives_per_game.to_json('../data/'+ players[0].nbaid() +'/20' + year[-2:] + 'drivespergame.json')
    print('Year '+ year + ' completed')

Year 2019-20 completed
Year 2020-21 completed
Year 2021-22 completed


Download data of usage, scoring and shooting

In [5]:
dict = {
    'trad': 'https://www.nba.com/stats/players/traditional/?sort=PTS&dir=-1&Season={0}&SeasonType=Regular%20Season&PerMode=Per100Possessions&CF=GP*GE*30',
    'adv': 'https://www.nba.com/stats/players/advanced/?sort=GP&dir=-1&Season={0}&SeasonType=Regular%20Season&CF=GP*GE*30',
    'scoring': 'https://www.nba.com/stats/players/scoring/?sort=MIN&dir=-1&Season={0}&SeasonType=Regular%20Season&CF=GP*GE*30',
    'misc': 'https://www.nba.com/stats/players/misc/?sort=GP&dir=-1&Season={0}&SeasonType=Regular%20Season&PerMode=Per100Possessions&CF=GP*GE*30',
    'usage': 'https://www.nba.com/stats/players/usage/?sort=USG_PCT&dir=-1&Season={0}&SeasonType=Regular%20Season&CF=GP*GE*30',
    'shooting': 'https://www.nba.com/stats/players/shooting/?Season={0}&SeasonType=Regular%20Season&DistanceRange=By%20Zone'
}

shooting_columns = ['PLAYER', 'TEAM', 'AGE', 'RA FGM', 'RA FGA', 'RA FG%',
                    'ITP FGM', 'ITP FGA', 'ITP FG%', 'MR FGM', 'MR FGA', 'MR FG%',
                    'LC3 FGM', 'LC3 FGA', 'LC3 FG%','RC3 FGM', 'RC3 FGA', 'RC3 FG%',
                    'C3 FGM', 'C3 FGA', 'C3 FG%', 'AB3 FGM', 'AB3 FGA', 'AB3 FG%']
# Uncomment next lines if first time to accept cookies and prevent errors
# driver.get('https://www.nba.com/stats/')
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'//*[@id="onetrust-accept-btn-handler"]'))).click()

seasons = list(set([s for player in players for s in player.nbaseasons()]))
s = ['2019-20']
for season in s:
    general_season = pd.DataFrame()
    for stat, url in dict.items():
        driver.get(url.format(season))
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))).click()
        driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]').click()
        html_table = driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table').get_attribute('outerHTML')
        stats_season = pd.read_html(html_table)[0]
        if 'Unnamed: 0' in stats_season.columns:
            stats_season = stats_season.drop('Unnamed: 0', axis=1)
        if stat == 'shooting':
            stats_season.columns = shooting_columns
            stats_season.dropna(axis=0, inplace=True)
        else:
            stats_season.dropna(axis=1, inplace=True)
        stats_season.columns = stats_season.columns.str.upper()
        if general_season.empty is True:
            general_season = stats_season
        else:
            general_season = pd.merge(general_season, stats_season, on='PLAYER', suffixes=('', '_y'))
            general_season = general_season.drop(general_season.filter(regex='_y$').columns.tolist(), axis=1)
        print(stat, season, 'completed')
    general_season.to_json('../data/byseason/' + season +'.json')


trad 2019-20 completed
adv 2019-20 completed
scoring 2019-20 completed
misc 2019-20 completed
usage 2019-20 completed
shooting 2019-20 completed


In [23]:
#driver.get('https://www.nba.com/stats/')
#WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'//*[@id="onetrust-accept-btn-handler"]'))).click()
dict = {
    'trad': 'https://www.nba.com/stats/players/traditional/?sort=PTS&dir=-1&Season={0}&SeasonType=Regular%20Season&PerMode=Per100Possessions&CF=GP*GE*30',
    'adv': 'https://www.nba.com/stats/players/advanced/?sort=GP&dir=-1&Season={0}&SeasonType=Regular%20Season&CF=GP*GE*30',
    'scoring': 'https://www.nba.com/stats/players/scoring/?sort=MIN&dir=-1&Season={0}&SeasonType=Regular%20Season&CF=GP*GE*30',
    'misc': 'https://www.nba.com/stats/players/misc/?sort=GP&dir=-1&Season={0}&SeasonType=Regular%20Season&PerMode=Per100Possessions&CF=GP*GE*30',
    'usage': 'https://www.nba.com/stats/players/usage/?sort=USG_PCT&dir=-1&Season={0}&SeasonType=Regular%20Season&CF=GP*GE*30',
    'shooting': 'https://www.nba.com/stats/players/shooting/?Season={0}&SeasonType=Regular%20Season&DistanceRange=By%20Zone'
}

shooting_columns = ['PLAYER', 'TEAM', 'AGE', 'RA FGM', 'RA FGA', 'RA FG%',
                    'ITP FGM', 'ITP FGA', 'ITP FG%', 'MR FGM', 'MR FGA', 'MR FG%',
                    'LC3 FGM', 'LC3 FGA', 'LC3 FG%','RC3 FGM', 'RC3 FGA', 'RC3 FG%',
                    'C3 FGM', 'C3 FGA', 'C3 FG%', 'AB3 FGM', 'AB3 FGA', 'AB3 FG%']
general_season = pd.DataFrame()
season = '2021-2022'
for stat, url in dict.items():
    driver.get(url.format(season))
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,'/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'))).click()
    driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select/option[1]').click()
    html_table = driver.find_element(By.XPATH, '/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table').get_attribute('outerHTML')
    stats_season = pd.read_html(html_table)[0]
    if 'Unnamed: 0' in stats_season.columns:
        stats_season = stats_season.drop('Unnamed: 0', axis=1)
    if stat == 'shooting':
        stats_season.columns = shooting_columns
        stats_season.dropna(axis=0, inplace=True)
    else:
        stats_season.dropna(axis=1, inplace=True)
    stats_season.columns = stats_season.columns.str.upper()
    if general_season.empty is True:
        general_season = stats_season
    else:
        general_season = pd.merge(general_season, stats_season, on='PLAYER', suffixes=('', '_y'))
        general_season = general_season.drop(general_season.filter(regex='_y$').columns.tolist(), axis=1)
    print(stat, season, 'completed')
#general_season.to_json('../data/byseason/' + season +'.json')

trad 2021-2022 completed
adv 2021-2022 completed
scoring 2021-2022 completed
misc 2021-2022 completed
usage 2021-2022 completed
shooting 2021-2022 completed


Shot charts data

In [15]:
for player in players:
    print('Starting with ' + player.name())
    for season in player.nbaseasons():
        # Create JSON request
        shot_json = shotchartdetail.ShotChartDetail(
                team_id = player.nbateamid(),
                player_id = player.nbaid(),
                context_measure_simple = 'FGA',
                season_nullable = season,
                season_type_all_star = 'Regular Season')
        # Load data into a Python dictionary
        shot_data = json.loads(shot_json.get_json())
        # Get the relevant data from our dictionary
        relevant_data = shot_data['resultSets'][0]
        # Get the headers and row data
        headers = relevant_data['headers']
        rows = relevant_data['rowSet']
        # Create pandas DataFrame
        player_data = pd.DataFrame(rows)
        player_data.columns = headers
        player_data.to_json('../data/' + player.nbaid() + '/' + season + '_shots.json')
        print(player.name() + ' ' + season + ' completed.')

Starting with Ja Morant
Ja Morant 2019-20 completed.
Ja Morant 2020-21 completed.
Ja Morant 2021-22 completed.
Starting with Derrick Rose
Derrick Rose 2008-09 completed.
Derrick Rose 2009-10 completed.
Derrick Rose 2010-11 completed.
Starting with Russell Westbrook
Russell Westbrook 2008-09 completed.
Russell Westbrook 2009-10 completed.
Russell Westbrook 2010-11 completed.
Starting with Allen Iverson
Allen Iverson 1996-97 completed.
Allen Iverson 1997-98 completed.
Allen Iverson 1998-99 completed.
